In [1]:
import torch
from transformers import AutoTokenizer, BertModel

In [2]:
import psycopg2
from setup import conn_params,MAX_CONTEXT,MODEL_NAME

In [3]:
import numpy as np

In [4]:
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
model=BertModel.from_pretrained(MODEL_NAME)

Some weights of BertModel were not initialized from the model checkpoint at avichr/Legal-heBERT and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# query code

In [5]:
def knn_query_postgresql(query_embedding, k=5):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT id,token_ids,embedding
                FROM snippets
                ORDER BY embedding <-> %s
                LIMIT %s;
            """, ( str(query_embedding), k))
            return cursor.fetchall()


In [10]:
def knn_query_best_snippets(query_embedding, k=5):
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cursor:
            cursor.execute("""
                WITH distance_per_snippet AS (
                    SELECT 
                        document_id, 
                        id, 
                        token_ids, 
                        embedding, 
                        embedding <-> %s AS distance
                    FROM 
                        snippets
                ),
                avg_distance_per_document AS (
                    SELECT 
                        document_id, 
                        AVG(distance) AS avg_distance
                    FROM 
                        distance_per_snippet
                    GROUP BY 
                        document_id
                ),
                top_documents AS (
                    SELECT 
                        document_id
                    FROM 
                        avg_distance_per_document
                    ORDER BY 
                        avg_distance
                    LIMIT %s
                ),
                best_snippets_per_document AS (
                    SELECT
                        dps.id,
                        dps.token_ids,
                        dps.embedding
                    FROM
                        distance_per_snippet dps
                    INNER JOIN top_documents td ON dps.document_id = td.document_id
                    ORDER BY 
                        dps.document_id, dps.distance
                    LIMIT %s
                )
                SELECT id, token_ids, embedding FROM best_snippets_per_document;
            """, (str(query_embedding), k, k))
            return cursor.fetchall()


# test

In [19]:
#test_text="שוחד, מרמה והפרת אמונים"
# test_text="""סימן ב׳: בגידה
# 97.
# פגיעה בריבונות המדינה או בשלמותה [ד/7]
# (א)
# מי שעשה, בכוונה לפגוע בריבונותה של המדינה, מעשה שיש בו כדי לפגוע בריבונותה, דינו – מיתה או מאסר עולם.
# (ב)
# מי שעשה, בכוונה ששטח כלשהו יצא מריבונותה של המדינה או ייכנס לריבונותה של מדינת חוץ, מעשה שיש בו כדי להביא לכך, דינו – מיתה או מאסר עולם.
# 98.
# גרם למלחמה [ד/8]
# מי שעשה, בכוונה להביא לידי פעולה צבאית נגד ישראל, מעשה שיש בו כדי להביא לכך, דינו – מאסר חמש עשרה שנה; התכוון לסייע לאויב, דינו – מיתה או מאסר עולם.
# 99.
# סיוע לאויב במלחמה [ד/9]
# (א)
# מי שעשה, בכוונה לסייע לאויב במלחמתו נגד ישראל, מעשה שיש בו כדי לסייעו לכך, דינו – מיתה או מאסר עולם.
# (ב)
# לענין סעיף זה, ”סיוע“ – לרבות מסירת ידיעה בכוונה שתגיע לידי האויב או ביודעין שהיא עשויה להגיע לידי האויב, ואין נפקא מינה שבעת מסירתה לא התנהלה מלחמה.
# 100.
# גילוי החלטה לבגוד [ד/10]
# מי שעשה מעשה המגלה אחת הכוונות האמורות בסעיפים 97, 98 או 99, דינו – מאסר עשר שנים.
# 101.
# שירות בכוחות האויב [ד/11]
# אזרח ישראלי וכל אדם אחר החייב בנאמנות למדינת ישראל, בין באשר הוא חייב בשירות בטחון לפי חוק שירות בטחון, תשי״ט–1959 [נוסח משולב], ובין באשר שירת בשירות המדינה, והוא שירת בכוחות המזויינים של האויב, דינו – מאסר חמש עשרה שנה.
# 102.
# סיוע לשבוי מלחמה [ד/12]
# (א)
# מי שסייע בידי שבוי מלחמה להימלט או לנסות להימלט ממקום כליאתו או מישראל, דינו – מאסר עשר שנים; התכוון לפגוע בבטחון המדינה, דינו – מאסר חמש עשרה שנה.
# (ב)
# מי שהתרשל בתפקידו למנוע בריחת שבוי מלחמה, דינו – מאסר שלוש שנים.
# 103.
# תעמולה תבוסנית [ד/13]
# מי שהפיץ בשעת לחימה, ובכוונה לגרום בהלה בציבור, ידיעות שיש בהן כדי לערער את רוח חיילי ישראל ותושביה בעמידתם בפני האויב, דינו – מאסר חמש שנים; התכוון לפגוע בבטחון המדינה, דינו – מאסר עשר שנים. """

# test_text="""סימן ה׳: עבירות מין
# 345.
# אינוס [תיקון: תשמ״ח־2, תש״ן־6, תש״ס־4, תשס״א־6]
# (א)
# הבועל אשה –
# (1)
# שלא בהסכמתה החופשית;
# (2)
# בהסכמת האשה, שהושגה במרמה לגבי מיהות העושה או מהות המעשה;
# (3)
# כשהאשה היא קטינה שטרם מלאו לה ארבע עשרה שנים, אף בהסכמתה; או
# (4)
# תוך ניצול מצב של חוסר הכרה בו שרויה האשה, או מצב אחר המונע ממנה לתת הסכמה חופשית;
# (5)
# תוך ניצול היותה חולת נפש או לקויה בשכלה, אם בשל מחלתה או בשל הליקוי בשכלה לא היתה הסכמתה לבעילה הסכמה חופשית;
# הרי הוא אונס ודינו – מאסר שש עשרה שנים.
# (ב)
# על אף האמור בסעיף קטן (א), דין האונס – מאסר עשרים שנים אם האינוס נעשה באחת מנסיבות אלה:
# (1)
# בקטינה שטרם מלאו לה שש עשרה שנים ובנסיבות האמורות בסעיף קטן (א)(1), (2), (4) או (5);
# (2)
# באיום בנשק חם או קר;
# (3)
# תוך גרימת חבלה גופנית או נפשית או הריון;
# (4)
# תוך התעללות באשה, לפני המעשה, בזמן המעשה או אחריו;
# (5)
# בנוכחות אחר או אחרים שחברו יחד עמו לביצוע האינוס בידי אחד או אחדים מהם.
# (ג)
# בסימן זה –
# ”בועל“ – המחדיר איבר מאיברי הגוף או חפץ לאיבר המין של האישה;
# ”קטין“ – (נמחקה). """ 
test_text="""
סימן ו׳: עבירות מרמה, סחיטה ועושק
414.
הגדרות [ח/1]
בסימן זה –
”דבר“ – מקרקעין, מיטלטלין, זכות וטובת הנאה;
”מרמה“ – טענת עובדה בענין שבעבר, בהווה או בעתיד, הנטענת בכתב, בעל פה או בהתנהגות, ואשר הטוען אותה יודע שאינה אמת או שאינו מאמין שהיא אמת; ו”לרמות“ – להביא אדם במרמה לידי מעשה או מחדל;
”מסמך“ – תעודה שבכתב, וכל אמצעי אחר, בין בכתב ובין בצורה אחרת, העשוי לשמש ראיה;
”זיוף“ – אחת מאלה:
(1)
עשיית מסמך הנחזה להיות את אשר איננו, והוא עשוי להטעות;
(2)
שינוי מסמך – לרבות הוספת פרט או השמטת פרט – בכוונה לרמות, או ללא סמכות כדין ובאופן הנחזה כאילו נעשה השינוי בסמכות כדין;
(3)
חתימת מסמך בשם פלוני ללא סמכות כדין, או בשם מדומה, באופן העשוי להיחזות כאילו נחתם המסמך בידי פלוני;
”שיק“ – כמשמעותו בפקודת השטרות, לרבות שטר חליפין משוך על אגודה שיתופית לאשראי שהוא בר פרעון עם דרישה;
”בנקאי“ – האדם שעליו משוך השיק.
415.
קבלת דבר במרמה [ח/2]
המקבל דבר במרמה, דינו – מאסר שלוש שנים, ואם נעברה העבירה בנסיבות מחמירות, דינו – מאסר חמש שנים.
416.
תחבולה [ח/3]
המקבל דבר בתחבולה או בניצול מכוון של טעות הזולת שאין בהם מרמה, דינו – מאסר שנתיים.
417.
כישוף [ח/4]
(א)
המתחזה לעשות מעשה כישוף בכוונה לקבל דבר, דינו – מאסר שנתיים; קיבל דבר בעד מעשה הכישוף או על פיו, דינו – מאסר שלוש שנים; לענין סעיף זה, ”כישוף“ – לרבות מעשה קוסם והגדת עתידות.
(ב)
הוראות סעיף קטן (א) לא יחולו על מעשה קוסם או הגדת עתידות שאינם חורגים מגדר שעשוע או בידור, והשעשוע או הבידור ניתנים ללא תמורה, או שתמורתם היא מחיר הכניסה בלבד למקום עריכתם.
418.
זיוף [ח/5]
המזייף מסמך, דינו – מאסר שנה; זייף מסמך בכוונה לקבל באמצעותו דבר, דינו – מאסר שלוש שנים; ואם נעברה העבירה בנסיבות מחמירות, דינו – מאסר חמש שנים.
419.
זיוף המשפיע על עסקאות [ח/6]
המזייף מסמך שיש בו ידיעה על אדם או על תאגיד בכוונה לרמות, דינו – מאסר שלוש שנים; אין נפקא מינה, לענין זה, אם האדם או התאגיד היו קיימים או לא ואם התאגיד עמד להיווסד אך לא נוסד.
420.
שימוש במסמך מזוייף [ח/7]
המגיש או מנפק מסמך מזוייף או משתמש בו בדרך אחרת, בידעו שהוא מזוייף, דינו כדין מזייף המסמך.
421.
זיוף בידי עובד הציבור [ח/8]
עובד הציבור המזייף מסמך שהוא ממונה על עשייתו או על שמירתו, או שיש לו גישה אליו בתוקף תפקידו, דינו – מאסר שלוש שנים עם או בלי קנס; עשה כן בכוונה לקבל דבר, דינו – מאסר חמש שנים עם או בלי קנס; קיבל דבר באמצעות מסמך מזוייף כאמור, דינו – מאסר שבע שנים עם או בלי קנס.
422.
שידולי מרמה [ח/9]
המשדל אדם במרמה לעשות מסמך או לחתום עליו או להשיג חתימת אדם אחר או חותמת על מסמך, דינו כדין מזייף ודין המסמך כדין מסמך מזוייף; המשדל אדם במרמה להשמיד מסמך ויש בכך כדי לגרום לו הפסד דבר, דינו כדין המקבל דבר במרמה; אין הוראות אלה באות לגרוע מהוראת כל דין אחר בענין שידול.
423.
רישום כוזב במסמכי תאגיד [ח/9א] [תיקון: [תש״ל], [תשל״ה]]
מייסד, מנהל, חבר או פקיד של תאגיד, הרושם, או גורם לרישום, פרט כוזב במסמך של התאגיד, בכוונה לרמות, או נמנע מלרשום בו פרט אשר היה עליו לרשמו, בכוונה לרמות, דינו – מאסר חמש שנים; לענין סעיף זה, וסעיפים 424 ו־425, ”תאגיד“ – לרבות תאגיד העומד להיווסד.
"""
#test_text="""תיק דניאל פרידמן נגד יואב גלנט: דיון בשאלת המינוי של יואב גלנט לרמטכ"ל צה"ל."""
print(test_text)


סימן ו׳: עבירות מרמה, סחיטה ועושק
414.
הגדרות [ח/1]
בסימן זה –
”דבר“ – מקרקעין, מיטלטלין, זכות וטובת הנאה;
”מרמה“ – טענת עובדה בענין שבעבר, בהווה או בעתיד, הנטענת בכתב, בעל פה או בהתנהגות, ואשר הטוען אותה יודע שאינה אמת או שאינו מאמין שהיא אמת; ו”לרמות“ – להביא אדם במרמה לידי מעשה או מחדל;
”מסמך“ – תעודה שבכתב, וכל אמצעי אחר, בין בכתב ובין בצורה אחרת, העשוי לשמש ראיה;
”זיוף“ – אחת מאלה:
(1)
עשיית מסמך הנחזה להיות את אשר איננו, והוא עשוי להטעות;
(2)
שינוי מסמך – לרבות הוספת פרט או השמטת פרט – בכוונה לרמות, או ללא סמכות כדין ובאופן הנחזה כאילו נעשה השינוי בסמכות כדין;
(3)
חתימת מסמך בשם פלוני ללא סמכות כדין, או בשם מדומה, באופן העשוי להיחזות כאילו נחתם המסמך בידי פלוני;
”שיק“ – כמשמעותו בפקודת השטרות, לרבות שטר חליפין משוך על אגודה שיתופית לאשראי שהוא בר פרעון עם דרישה;
”בנקאי“ – האדם שעליו משוך השיק.
415.
קבלת דבר במרמה [ח/2]
המקבל דבר במרמה, דינו – מאסר שלוש שנים, ואם נעברה העבירה בנסיבות מחמירות, דינו – מאסר חמש שנים.
416.
תחבולה [ח/3]
המקבל דבר בתחבולה או בניצול מכוון של טעות הזולת 

In [20]:
with torch.no_grad():
    inputs=tokenizer(test_text,return_tensors='pt',truncation=True,max_length=model.config.max_position_embeddings)
    inputs={k:v.to(model.device) for k,v in inputs.items()}
    key=model(**inputs).last_hidden_state.mean([1]).cpu().tolist()

In [21]:
#ans=knn_query_postgresql(key[0],3)
ans=knn_query_best_snippets(key[0],7)

In [22]:
texts=[tokenizer.decode(l[1]) for l in ans]
print(len(texts)-len(set(texts)))
texts

0


['[CLS] בית המשפט העליון בשבתו כבית משפט לערעורים פליליים ע " פ 97 / 2085 בפני : כבוד השופט א\'גולדברג כבוד השופט י\'קדמי כבוד השופט י\'זמיר המערער : מוחמד בן ג\'מיל שורפי נגד המשיב ה : מדינת ישראל ערעור על פסק דינו של בית המשפט המחוזי בתל - אביב - יפו מיום23. 2. 97 בת " פ 521 / 95 שניתן על ידי כבוד השופטת ע. קפלן - הגלר תאריך הישיבה : י " א באייר התשנ " ז ( 18. 05. 97 ) בשם המערער : עו " ד ח\'כבוב בשם המשיב ה : עו " ד מ\'שילה פסק - דין חרף טענותיו הממצות של בא כוחו המלומד של המערער, אין בידינו לקבל את הטענה כי אין ראיה משכנעת כי המערער היה עם מניעי בסמוך למקום המטמון של הסם. ראיה כזאת קיימת אם בדרך האלימינציה ואם בדרך של ראיות ישירות. משהגיע המערער עם מניעי עד סמוך למקום הסם, והאחרון אף הרים את מכסה העץ שכיסה על הסם, לא ניתן להסיק אלא זאת שלמערער היתה נגיעה לסם, בהעדר הסבר אחר מצדו לסיבת בואו והמצאותו במקום. הסבר כזה לא נתן המערער. המערער הורשע בנסיון להחזיק סם מסוכן בכמות 76. 3 גרם הרואין, היא כמות הסם שנמצאה במחבוא טרם שהמשטרה המירה בהחבא את הסם בחומר אחר, והציבה תצפיות במקום. משהוכ

In [23]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  